In [1]:
import pdfplumber

with pdfplumber.open("facture_client.pdf") as pdf:
    page = pdf.pages[0]
    lines = page.lines  # chaque ligne a x0, y0, x1, y1
    for line in lines:
        print(line)


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


{'x0': 530.25, 'y0': 147.5, 'x1': 530.25, 'y1': 561.5, 'width': 0.0, 'height': 414.0, 'pts': [(530.25, 280.5), (530.25, 694.5)], 'linewidth': 0.75, 'stroke': True, 'fill': False, 'evenodd': False, 'stroking_color': (0, 0, 0), 'non_stroking_color': (1, 1, 1), 'mcid': None, 'tag': None, 'object_type': 'line', 'page_number': 1, 'stroking_pattern': None, 'non_stroking_pattern': None, 'path': [('m', (530.25, 280.5)), ('l', (530.25, 694.5))], 'dash': ([], 0), 'top': 280.5, 'bottom': 694.5, 'doctop': 280.5}
{'x0': 561.75, 'y0': 147.5, 'x1': 561.75, 'y1': 561.5, 'width': 0.0, 'height': 414.0, 'pts': [(561.75, 280.5), (561.75, 694.5)], 'linewidth': 0.75, 'stroke': True, 'fill': False, 'evenodd': False, 'stroking_color': (0, 0, 0), 'non_stroking_color': (1, 1, 1), 'mcid': None, 'tag': None, 'object_type': 'line', 'page_number': 1, 'stroking_pattern': None, 'non_stroking_pattern': None, 'path': [('m', (561.75, 280.5)), ('l', (561.75, 694.5))], 'dash': ([], 0), 'top': 280.5, 'bottom': 694.5, 'doct

In [3]:
import os
# import openai
import pdfplumber
import warnings
import requests
warnings.filterwarnings("ignore", category=UserWarning, module="pdfplumber")
import re

def extract_text_from_pdf(pdf_path):
    """ Extracts text from a PDF file """
    with pdfplumber.open(pdf_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            full_text += page.extract_text() + "\n\n"
    return full_text

def enlever_caracteres_speciaux(texte):
    """
    Enlève les caractères spéciaux d'un texte en conservant les lettres, chiffres et espaces.
    """
    pattern = r'[^a-zA-Z0-9\s€$.,:;!?%()\-_/éèàùçâêîôûëïüöñ]'
    texte_nettoye = re.sub(pattern, '', texte)
    return texte_nettoye


def convert_to_docling(text):
    """ Converts raw text to a DOCLING structure """
    lines = text.split("\n")
    docling = {
        "header": [],
        "body": [],
        "footer": []
    }

    section = "header"

    for line in lines:
        if "Total" in line or "TVA" in line:
            section = "footer"
        elif any(keyword in line for keyword in ["Article", "Quantité", "Prix"]):
            section = "body"
        docling[section].append(line.strip())

    return docling

def docling_to_markdown(docling):
    """ Converts DOCLING structure to Markdown """
    markdown = "# Voici une facture, Réponds moi avec cette structure :\n{{\n    \"entreprise\": \"\",\n    \"numero_facture_ou_piece\": \"\",\n    \"date\": \"\",\n    \"montant_TTC \": \"\",\n  \"montant_TVA\": \"\",\n  \"nom_client\": \"\",\n  \"code_client\": \"\"\n}}\n\n Voici le contenu OCR extrait :\n\n"

    if docling["header"]:
        markdown += "## En-tête\n\n" + "\n".join(docling["header"]) + "\n\n"

    if docling["body"]:
        markdown += "### Détails des articles\n\n"
        markdown += "| Article | Quantité | Prix Unitaire | Total |\n"
        markdown += "|---------|----------|--------------|-------|\n"
        for line in docling["body"]:
            parts = line.split()
            if len(parts) >= 4:
                markdown += f"| {' '.join(parts[:-3])} | {parts[-3]} | {parts[-2]} | {parts[-1]} |\n"
        markdown += "\n"

    if docling["footer"]:
        markdown += "### Pied de page\n\n" + "\n".join(docling["footer"]) + "\n\n"

    return markdown

def query_ollama(model, prompt):

    # 7. Appel à Ollama
    response = requests.post("http://localhost:11434/api/generate", json={
        "model": model,
        "prompt": prompt,
        "stream": False
    })

    return response.json()["response"].strip()


def main(pdf_path, model = 'mistral'):
    text = extract_text_from_pdf(pdf_path)
    text = enlever_caracteres_speciaux(text)
    docling = convert_to_docling(text)
    markdown = docling_to_markdown(docling)
    print(markdown)
    print(model, "model")
    response = query_ollama(model,markdown)
    print("LLM Response:\n", response)






In [4]:
pdf_path = "facture_client.pdf"  # Update with your PDF path
main(pdf_path)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


# Voici une facture, Réponds moi avec cette structure :
{{
    "entreprise": "",
    "numero_facture_ou_piece": "",
    "date": "",
    "montant_TTC ": "",
  "montant_TVA": "",
  "nom_client": "",
  "code_client": ""
}}

 Voici le contenu OCR extrait :

## En-tête

Facture N FA42401302
Hopital privé Wallerstein
14 boulevard JAVAL
33740 ARES
Date Numéro F Client Délai de règlement Mode de règlement
30/09/2024 FA42401302 1 A403301860 A réception facture Chèque

### Pied de page

Article Désignation Quantité Prix Unitaire Montant TVA
Facture davancement Juillet 2024
IGPREST041 EIG Pack de démarrage 3,50 500,00 1 750,00 3
- Suivi de Projets (réunions, suivi des chantiers...)
Réalisations :
- Finalisation création base
- Gestion des Emplois
- Régime Stagiaire
- Préparation de limport des Salariés (via DSN)
- MAJ des Taux AT  Médecine
- Suivi de projet
Intervenant : Florence ESTEVES
Facture(s) en attente de paiement au 18/04/2025 :
- Facture FA42401149 du 31/07/2024 de 600,00 €
- Facture FA4

In [4]:
pdf_path = "facture_fo_1.pdf"  # Update with your PDF path
main(pdf_path)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


LLM Response:
 {
      "entreprise": "ATHENA GLOBAL SERVICES",
      "numero_facture_ou_piece": "FACTURE EIG",
      "date": "23/04/2025",
      "montant_TTC ": "104,96 €",
      "montant_TVA": "17,49 €",
      "nom_client": "Votre identification TVA : FR01382879625",
      "code_client": "A7577 2504005614"
   }


In [5]:
pdf_path = "facture_fo_2.pdf"  # Update with your PDF path
main(pdf_path)

CropBox missing from /Page, defaulting to MediaBox


CropBox missing from /Page, defaulting to MediaBox


LLM Response:
 {{
      "entreprise": "CarolVooyages",
      "numero_facture_ou_piece": "FACTURE N F-003 0019631",
      "date": "23/04/2025",
      "montant_TTC ": "339.00",
      "montant_TVA": "2.00",
      "nom_client": "MR BERSON GAETAN",
      "code_client": "1009976"
   }}


In [6]:
pdf_path = "facture_fo_3.pdf"  # Update with your PDF path
main(pdf_path)

CropBox missing from /Page, defaulting to MediaBox


CropBox missing from /Page, defaulting to MediaBox


LLM Response:
 {
    "entreprise": "IMA PROTECT",
    "numero_facture_ou_piece": "FACTURE",
    "date": "01/05/25",
    "montant_TTC ": "46,44",
    "montant_TVA": "7,74",
    "nom_client": "BUON CATHERINE",
    "code_client": "20153341043"
  }


In [7]:
pdf_path = "facture_fo_4.pdf"  # Update with your PDF path
main(pdf_path)

LLM Response:
 {
    "entreprise": "Etix Everywhere Nord EIG SAS",
    "numero_facture_ou_piece": "FCNO2504-04041",
    "date": "01/04/2025",
    "montant_TTC": "4530,00",
    "montant_TVA": "755,00",
    "nom_client": "CU2307-00168",
    "code_client": "H19039_EL3"
   }


In [8]:
pdf_path = "facture_fo_5.pdf"  # Update with your PDF path
main(pdf_path)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


LLM Response:
 {
    "entreprise": "DIAC LOCATION",
    "numero_facture_ou_piece": "P50470642",
    "date": "01/04/2025",
    "montant_TTC": "778,21 EUR",
    "montant_TVA": "129,69 EUR",
    "nom_client": "241780520 FACTURE",
    "code_client": "241780520"
}


In [9]:
pdf_path = "facture_fo_6.pdf"  # Update with your PDF path
main(pdf_path)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


LLM Response:
 {{
      "entreprise": "Entreprise Non Identifiée",
      "numero_facture_ou_piece": "Non Spécifié",
      "date": "Date non identifiable",
      "montant_TTC ": "Montant TTC non spécifié",
      "montant_TVA": "Montant TVA non spécifié",
      "nom_client": "Client Non Identifié",
      "code_client": "Code Client Non Spécifié"
   }}


In [4]:
import base64
import requests

# Encode l'image en base64
with open("facture.png", "rb") as image_file:
    image_base64 = base64.b64encode(image_file.read()).decode("utf-8")

# Requête vers Ollama avec image
response = requests.post("http://localhost:11434/api/generate", json={
    "model": "llava:13b",  # ou autre modèle multimodal
    "prompt": "Donne moi le numéro de facture et le montant total",
    "images": [image_base64],  # liste de chaînes base64
    "stream": False
})

print(response.json()["response"])

 Le numéro de la facture dans l'image est "N° F.17320" et le montant total est indiqué comme étant "TOTAL FACTURE : 159,76 €".


In [ ]:
pdf_path = "facture_client.pdf"  # Update with your PDF path
main(pdf_path, model = 'llama3:70b')

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


llama3:70b model


: 

In [4]:
pdf_path = "facture_client.pdf"  # Update with your PDF path
main(pdf_path, model = 'phi3:mini')

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


phi3:mini model
LLM Response:
 {{
    "entreprise": "Hopital privé Wallerstein",
    "numero_facture_ou_piece": "FA42401302",
    "date": "30/09/2024",
    "montant_TTC ": "38 469,32 €",
    "montant_TVA": "2 100,00 €" ,
    "nom_client": "",
    "code_client": ""
}}


In [5]:
pdf_path = "facture_client.pdf"  # Update with your PDF path
main(pdf_path, model = 'mixtral:8x7b')

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


mixtral:8x7b model
LLM Response:
 ici est la structure avec les informations extraites de l'OCR :

```json
{
    "entreprise": "Hopital privé Wallerstein",
    "numero_facture_ou_piece": "FA42401302",
    "date": "30/09/2024",
    "montant_TTC ": "2100.00",
    "montant_TVA": "0.00",
    "nom_client": "",
    "code_client": "A403301860"
}
```

Note : Les informations sur le nom du client sont manquantes dans la facture fournie.
